# Clustering Crypto

In [263]:
# Initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [321]:
# Load data
filepath = "Resources/crypto_data.csv"
df = pd.read_csv(filepath)
df

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...,...
1247,XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
1248,DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
1249,GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
1250,OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


In [322]:
# Keep all the cryptocurrencies that are being traded.
df["IsTrading"].value_counts()

df = df.loc[df["IsTrading"]==True]
df

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...,...
1243,SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
1244,UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
1245,BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [323]:
# Keep all the cryptocurrencies that have a working algorithm.
df["Algorithm"].isnull().sum()

#all cryptocurrencies seem to have an algorithm

0

In [324]:
# Remove the "IsTrading" column. 
df = df.drop(columns="IsTrading")
df

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,PoW,NaN,611000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
1243,SERO,Super Zero,Ethash,PoW,NaN,1000000000
1244,UOS,UOS,SHA-256,DPoI,NaN,1000000000
1245,BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [325]:
# Checking null values
df.isnull().sum()

# Remove rows that have at least 1 null value.
df.dropna()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...,...
1238,ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [326]:
# Keep the rows where coins are mined.
print(df["TotalCoinsMined"].value_counts())
df = df.loc[df["TotalCoinsMined"]>0]
df

0.000000e+00    152
1.000000e+09      7
1.000000e+08      7
2.000000e+07      2
2.500000e+07      2
               ... 
8.345040e+07      1
1.666282e+07      1
1.195525e+06      1
1.345549e+08      1
1.283270e+05      1
Name: TotalCoinsMined, Length: 520, dtype: int64


,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...,...
1238,ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [327]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names_df = df[["CoinName", "Unnamed: 0"]].set_index("Unnamed: 0")
crypto_names_df.rename_axis(None, inplace=True)
crypto_names_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [328]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
df = df.drop(columns=["CoinName"])
df

,Unnamed: 0,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,Scrypt,PoW/PoS,4.199995e+01,42
2,404,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
1238,ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
1242,GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,BDX,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Equihash,PoW,7.296538e+06,21000000


In [329]:
# Set coin name abreviation as index
df = df.set_index("Unnamed: 0")
df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610


In [330]:
# Convert TotalCoinSupply to float type since it imported as an object
df["TotalCoinSupply"]=pd.to_numeric(df["TotalCoinSupply"])
df.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply    float64
dtype: object

In [331]:
# Use get_dummies() to create variables for text features.
df.dtypes
X = pd.get_dummies(df, columns=["Algorithm", "ProofType"])
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2.000000e+09,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,2.500000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1.400223e+09,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [332]:
# Standardize the data with StandardScaler().
scaled_data = StandardScaler().fit_transform(X)
scaled_data

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [333]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)

In [334]:
# Create a DataFrame with the three principal components.
crypto_pca = pca.fit_transform(scaled_data)

# Make dataframe from pca data, name columns and set index as cryptocurrency abbreviations
pcs_df = pd.DataFrame(crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=X.index)

#Get rid of index name
pcs_df.rename_axis(None, inplace=True)

pcs_df 



,PC 1,PC 2,PC 3
42,-0.332732,1.014437,-0.573406
404,-0.316064,1.014517,-0.573739
1337,2.305949,1.678979,-0.626740
BTC,-0.151869,-1.273693,0.205304
ETH,-0.151151,-2.016259,0.412690
...,...,...,...
ZEPH,2.464187,0.892719,0.052037
GAP,-0.330777,1.014308,-0.573428
BDX,0.321976,-2.301728,0.408003
ZEN,-0.133248,-2.090950,0.421591


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [335]:
# Create an elbow curve to find the best value for K.

k = list(range(1,11))
inertia = []

for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
    
elbow_df = pd.DataFrame({
    "K": k,
    "Inertia": inertia
})

elbow_df.hvplot.line(x="K", y="Inertia")

:Curve   [K]   (Inertia)

Running K-Means with `k=4`

In [336]:
# Initialize the K-Means model.
km = KMeans(n_clusters=4)

# Fit the model
km.fit(pcs_df)

# Predict clusters
predictions = km.predict(pcs_df)
predictions

array([1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,

In [344]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([df, pcs_df], axis=1)
clustered_df

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = crypto_names_df

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = km.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.332732,1.014437,-0.573406,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.316064,1.014517,-0.573739,404Coin,1
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.305949,1.678979,-0.626740,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.151869,-1.273693,0.205304,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.151151,-2.016259,0.412690,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,8.400000e+07,-0.163994,-1.162566,-0.017838,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,2.200000e+07,-0.399334,1.339559,-0.523341,Dash,1
XMR,CryptoNight-V7,PoW,1.720114e+07,0.000000e+00,-0.155162,-2.185392,0.385169,Monero,0
ETC,Ethash,PoW,1.133597e+08,2.100000e+08,-0.149592,-2.016369,0.412674,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,2.100000e+07,-0.133247,-2.090950,0.421591,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [349]:
# Creating a 3D-Scatter with the PCA data and the clusters
px.scatter_3d(data_frame=clustered_df, x="PC 1", y="PC 2", z="PC 3", hover_name="CoinName", hover_data=["Algorithm"], color="Class", symbol="Class")


In [350]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinsMined", "TotalCoinSupply", "Class"], sortable=True, selectable=True)



:Table   [CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,Class]

In [352]:
# Print the total number of tradable cryptocurrencies.
count = clustered_df["CoinName"].count()
f"There are {count} tradable cryptocurrencies"

'There are 532 tradable cryptocurrencies'

In [357]:
# Scaling data to create the scatter plot with tradable cryptocurrencies. 
pre_scaled_data = clustered_df[["TotalCoinSupply", "TotalCoinsMined"]]

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(pre_scaled_data)
scaled_data

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [361]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
scaled_data_df = pd.DataFrame(scaled_data, columns=["TotalCoinSupplyScaled", "TotalCoinsMinedScaled"]).set_index(clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
scaled_data_df["CoinName"] = clustered_df["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
scaled_data_df["Class"] = clustered_df["Class"]

scaled_data_df


,TotalCoinSupplyScaled,TotalCoinsMinedScaled,CoinName,Class
42,4.200000e-11,0.000000e+00,42 Coin,1
404,5.320000e-04,1.065855e-03,404Coin,1
1337,3.141593e-01,2.957551e-02,EliteCoin,1
BTC,2.100000e-05,1.810842e-05,Bitcoin,0
ETH,0.000000e+00,1.087731e-04,Ethereum,0
...,...,...,...,...
ZEPH,2.000000e-03,2.020225e-03,ZEPHYR,1
GAP,2.500000e-04,1.508199e-05,Gapcoin,1
BDX,1.400223e-03,9.901351e-04,Beldex,0
ZEN,2.100000e-05,7.370282e-06,Horizen,0


In [363]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
scaled_data_df.hvplot.scatter(x="TotalCoinsMinedScaled", y="TotalCoinSupplyScaled", by="Class", hover_cols=["CoinName"])


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMinedScaled]   (TotalCoinSupplyScaled,CoinName)